In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats

import datetime

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import datetime

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score, precision_score, recall_score, roc_auc_score, roc_curve


In [18]:
df = pd.read_csv(r'processed_dataset.csv')

In [5]:
#Scaling the Data¶
#Checking feature importance
dfnew=df[['Latitude','Longitude','day','month','1st_Road_Number','year','Day_of_Week','Accident_Severity']]

In [6]:
df.columns

Index(['Longitude', 'Latitude', 'Police_Force', 'Accident_Severity',
       'Number_of_Vehicles', 'Number_of_Casualties', 'Day_of_Week',
       '1st_Road_Class', '1st_Road_Number', 'Road_Type', 'Speed_limit',
       'Junction_Control', '2nd_Road_Class', '2nd_Road_Number',
       'Pedestrian_Crossing-Human_Control',
       'Pedestrian_Crossing-Physical_Facilities', 'Light_Conditions',
       'Weather_Conditions', 'Road_Surface_Conditions',
       'Special_Conditions_at_Site', 'Carriageway_Hazards',
       'Urban_or_Rural_Area', 'Did_Police_Officer_Attend_Scene_of_Accident',
       'day', 'month', 'year'],
      dtype='object')

In [7]:
dfnew.head()

,Latitude,Longitude,day,month,1st_Road_Number,year,Day_of_Week,Accident_Severity
0,51.489096,-0.191170,1,1,3218,1970,3,1
1,51.520075,-0.211708,1,1,450,1970,4,2
2,51.525301,-0.206458,1,1,0,1970,5,2
3,51.482442,-0.173862,1,1,3220,1970,6,2
4,51.495752,-0.156618,1,1,0,1970,2,2


In [8]:
from sklearn.preprocessing import StandardScaler
features = [feature for feature in dfnew.columns if feature!='Accident_Severity']
x = dfnew.iloc[0:50000, :-1]
y = dfnew.iloc[0:50000,[-1]]
x = StandardScaler().fit_transform(x)


In [9]:
#Using SMOTE to Treat imbalenced Dataset¶
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
oversample = RandomOverSampler()
x, y = oversample.fit_resample(x, y)

In [10]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
import joblib
import warnings
warnings.filterwarnings("ignore")

In [12]:
from catboost import CatBoostClassifier
import time
from sklearn.metrics import accuracy_score

In [13]:
import pandas as pd
import time
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

# -----------------------------------------------------------------------------
## 1. TRAIN BASE MODELS (Assuming Best Parameters Found from Tuning)
# -----------------------------------------------------------------------------

# Optimized XGBoost (Base Model 1)
# Using high n_estimators and low learning_rate for better performance
best_xgb_clf = XGBClassifier(
    objective='multi:softmax',
    n_estimators=1000,
    max_depth=12,
    learning_rate=0.05,
    gamma=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    use_label_encoder=False,
    eval_metric='mlogloss',
    random_state=42,
    tree_method='hist'
)

# Optimized CatBoost (Base Model 2)
best_cb_clf = CatBoostClassifier(
    iterations=800,
    learning_rate=0.08,
    depth=10,
    l2_leaf_reg=5, # Higher regularization for stability
    loss_function='MultiClass',
    random_seed=42,
    verbose=0,
    thread_count=4
)

print("\n--- Training Base Models (XGBoost & CatBoost) ---")
best_xgb_clf.fit(x_train, y_train.values.ravel())
best_cb_clf.fit(x_train, y_train.values.ravel())

# -----------------------------------------------------------------------------
## 2. STACKING ENSEMBLE TRAINING
# -----------------------------------------------------------------------------

# Setup Base Models (Level 0)
estimators = [
    ('xgb', best_xgb_clf), 
    ('cat', best_cb_clf)
]

# Setup Meta Model (Level 1) - Used to combine base predictions
meta_clf = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=500, n_jobs=4, random_state=42)

# Create Stacking Classifier
stacked_clf = StackingClassifier(
    estimators=estimators,
    final_estimator=meta_clf,
    cv=3, # Use 3-fold cross-validation for stability
    n_jobs=4 
)

print("--- Training Stacking Ensemble ---")
start_time = time.time()

# Train the Stacked Ensemble
stacked_clf.fit(x_train, y_train.values.ravel())

end_time = time.time()
print(f"Time taken for Stacking fit: {(end_time - start_time):.2f} seconds")

# -----------------------------------------------------------------------------
## 3. FINAL PREDICTION AND SCORING
# -----------------------------------------------------------------------------

# Make predictions using the final stacked model
y_pred_stacked = stacked_clf.predict(x_test)
stacked_accuracy = accuracy_score(y_test, y_pred_stacked)

print(f"\n🔥 STACKED ENSEMBLE FINAL ACCURACY: {stacked_accuracy:.4f}")

# Final Classification Report
print("\nClassification Report (Stacked Ensemble)")
print(classification_report(y_test, y_pred_stacked, target_names=['Severe (0)', 'Serious (1)', 'Slight (2)']))


--- Training Base Models (XGBoost & CatBoost) ---
--- Training Stacking Ensemble ---
Time taken for Stacking fit: 187.64 seconds

🔥 STACKED ENSEMBLE FINAL ACCURACY: 0.9378

Classification Report (Stacked Ensemble)
              precision    recall  f1-score   support

  Severe (0)       0.99      1.00      0.99      8682
 Serious (1)       0.88      0.95      0.91      8585
  Slight (2)       0.95      0.87      0.91      8801

    accuracy                           0.94     26068
   macro avg       0.94      0.94      0.94     26068
weighted avg       0.94      0.94      0.94     26068



In [19]:
import joblib

# Save stacking model
joblib.dump(stacked_clf, "stacked_model.joblib")

# Save XGBoost using native format (BEST)
best_xgb_clf.save_model("xgb_model.json")

# Save CatBoost using native format (MANDATORY)
best_cb_clf.save_model("catboost_model.cbm")

print("✅ Models saved correctly")


✅ Models saved correctly
